## seq2seq

seq2seq(sequence to sequence)는 입력 시퀀스(input sequence)에 대한 출력 시퀀스(output sequence)를 만들기 위한 모델입니다. seq2seq는 품사 판별과 같은 시퀀스 레이블링(sequence labeling)과는 차이가 있습니다. 시퀀스 레이블링이란 입력 단어가 $x_1$, $x_2$, ... , $x_n$ 이라면 출력은 $y_1$, $y_2$, ... , $y_n$ 이 되는 형태입니다. 즉, 입력과 출력에 대한 문자열(sequence)이 같습니다. 하지만 seq2seq은 품사 판별보다는 `번역`에 초점을 둔 모델입니다. 번역은 입력 시퀀스의 $x_{1:n}$과 의미가 동일한 출력 시퀀스 $y_{1:m}$을 만드는 것이며, $x_i$, $y_i$ 간의 관계는 중요하지 않습니다. 그리고 각 시퀀스 길이도 서로 다를 수 있습니다.

![](../Static/568.jpg)

그럼 지금부터 seq2seq를 파이토치로 구현해 보겠습니다. 영어를 프랑스어로 번역하는 예제입니다. 이 예제는 파이토치 튜토리얼에 게시된 코드를 수정한 것입니다. 튜토리얼 코드와 비교하면서 학습해도 좋습니다.

In [16]:
from __future__ import unicode_literals, print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


* `__future__` 는 구 버전에서 상위 버전의 기능을 이용해야 할 때 사용합니다. 모듈을 import 하여 사용하는 것처럼 __future__를 import 하여 상위 버전의 기능을 사용합니다. 물론 최신 버전의 파이토치를 사용하는 경우에는 필요하지 않습니다. 예제는 사용 방법을 익히기 위해 추가해 두었습니다.

* re모듈은 `정규표현식`을 사용하고자 할 때 씁니다.

데이터셋은 타토에바 프로젝트 중에서 영어-프랑스어 파일을 사용합니다. 다음 URL에서 다양한 언어에 대한 것들을 제공하기 있기 때문에 예제에서 사용하는 영어-프랑스어 외에도 다른 언어를 내려받아 사용할 수 있습니다. 물론 영어-한국어도 제공합니다.

http://www.manythings.org/anki/

파이토치에서는 문장 그대로 사용할 수 없습니다. 문장을 단어로 분할하고 벡터(vector)로 변환해야 합니다.

In [17]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20


class Lang: # 딕셔너리를 만들기 위한 클래스
    def __init__(self): # 단어의 인덱스를 저장하기 위한 컨테이너 초기화
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"} # sos(문장의 시작), eos(문장의 끝)
        self.n_words = 2 #sos와 eos에 대한 카운트

    def addSentence(self, sentence): # 문장을 단어 단위로 분리한 후 컨테이너(word)에 추가
       for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word): # 컨테이너에 단어가 없다면 추가되고, 있다면 카운트를 업데이트
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


데이터셋으로 사용된 데이터는 앞에서 살펴보았듯이 탭(tab)으로 구성된 text 파일입니다. 따라서 데이터는 판다스로 불러온 후 정규화해야합니다.

In [18]:
def normalizeString(df, lang):
    sentence = df[lang].str.lower() # 소문자로 변환
    sentence = sentence.str.replace('[^A-Za-z\s]+', ' ')
    sentence = sentence.str.normalize('NFD') # 유니코드 정규화 방식
    sentence = sentence.str.encode('ascii', errors='ignore').str.decode('utf-8') # 유니코드를 ascii로 전환
    return sentence

def read_sentence(df, lang1, lang2):
    sentence1 = normalizeString(df, lang1) # 데이터의 첫 번째 열 (영어)
    sentence2 = normalizeString(df, lang2) # 데이터의 두 번째 열 (선택한 언어)
    return sentence1, sentence2

def read_file(loc, lang1, lang2):
    df = pd.read_csv(loc, delimiter='\t', header=None, names=[lang1, lang2,'cc'])
    df.drop(axis=1, columns='cc', inplace=True)
    return df

def process_data(lang1, lang2): # 데이터셋 불러오기
    df = read_file('./fra-eng/fra.txt', lang1, lang2)
    sentence1, sentence2 = read_sentence(df, lang1, lang2)

    input_lang = Lang()
    output_lang = Lang()
    pairs = []
    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]]
            input_lang.addSentence(sentence1[i])
            output_lang.addSentence(sentence2[i])
            pairs.append(full)
    
    return input_lang, output_lang, pairs







* df = pd.read_csv(loc, delimiter='\t', header=None, names=[lang1, lang2,'cc'])
    * loc : 예제에서 사용할 데이터셋
    * delimiter : CSV파일의 데이터가 어떤 형태(\t, ' ','+' 등)로 나뉘었는지 의미합니다. 데이터를 " " 묶음으로 처리할 때 사용합니다. 예를 들어 "Sure, I'm OK" 처럼 문자열에 콤마가 포함되어 있을 경우 "Sure"와 "I'm OK"로 나뉘는데, 이를 방지할 수 있습니다. 즉, 하나의 문장이 분할되지 않고 그대로 사용하고 싶을 때 유용합니다.
    * header : 일반적으로 데이터셋의 첫 번째 행을 header(열 이름)로 지정해서 사용하게 되는데, 불러올 데이터에 header가 없을 경우 `header=None` 옵션을 사용합니다.
    * names : 열 이름을 리스트 형태로 입력합니다. 데이터셋의 총 세 개의 열이 있기 때문에 lang1, lang2 그리고 저작권열인 cc를 입력하고 cc를 드랍합니다.

이제 데이터 쌍(paris)을 텐서로 변환해야 합니다. 계속 이야기하지만 파이토치의 네트워크는 텐서 유형의 데이터만 인식하기 때문에 매우 중요한 작업입니다. 이 작업은 중요한 또 다른 이유는 지금 진행하고 있는 데이터셋이 문장이기 때문입니다. 따라서 문장의 모든 끝에 입력이 완료되었음을 네트워크에 알려 주어야 하는데, 그것이 토큰입니다.

In [19]:
def indexesFromSentence(lang, sentence): # 문장을 단어로 분리하고 인덱스를 반환
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence): # 딕셔너리에서 단어에 대한 인덱스를 가져오고 문장끝에 토큰을 추가
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(input_lang, output_lang, pair): # 입력과 출력 문장을 텐서로 변환하여 반환
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


파이토치에서 seq2seq 모델을 사용하기 위해서는 먼저 인코더와 디코더를 정의해야 합니다.
입력(영어) 문장이 인코더로 주입되면 디코더(프랑스어)로 번역되어 출력됩니다. 인코더와 디코더를 이용하면 문장의 번역뿐만 아니라 다음 입력을 예측하는 것도 가능합니다. 이때 각 입력 문장의 끝에는 문장의 끝을 알리는 토큰이 할당됩니다.

![](../Static/572.jpg)

`인코더`는 입력 문장을 단어별로 순서대로 인코딩을 하게 되며, 문장의 끝을 표시하는 토큰이 붙습니다. 또한, 인코더는 `임베딩 계층`과 `GRU 계층`으로 구성됩니다.

![](../Static/573.jpg)

임베딩 계층은 입력에 대한 임베딩 결과가 저장되어 있는 딕셔너리를 조회하는 테이블과도 같습니다. 이후 GRU 계층과 연결되는데, GRU 계층은 연속하여 들어오는 입력을 계산합니다. 또한, 이전 계층의 은닉 상태를 계산한 후 망각 게이트와 업데이트 게이트를 갱신합니다.



In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
        super(Encoder, self).__init__()
        self.input_dim = input_dim # 인코더에서 사용할 입력층
        self.embbed_dim = embbed_dim # 인코더에서 사용할 임베딩 계층
        self.hidden_dim = hidden_dim # 인코더에서 사용할 은닉층(이전 은닉층)
        self.num_layers = num_layers # 인코더에서 사용할 GRU의 계층 개수
        self.embedding = nn.Embedding(input_dim, self.embbed_dim) # 임베딩 계층 초기화
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers) # 임베딩 차원, 은닉층 차원, GRU의 계층 개수를 이용하여 GRU 계층을 초기화

    def forward(self, src):
        embedded = self.embedding(src).view(1, 1, -1) # 임베딩 처리
        outputs, hidden = self.gru(embedded) # 임베딩 결과를 GRU 모델에 적용
        return outputs, hidden


`디코더`는 인코더 출력을 디코딩하여 다음 출력을 예측합니다. 디코더는 임베딩 계층, GRU 계층, 선형(linear) 계층으로 구성됩니다.

![](../Static/574.jpg)

임베딩 계층에서는 출력을 위해 딕셔너리를 조회할 테이블을 만들며, GRU 계층에서는 다음단어를 예측하기 위한 확률을 계산합니다. 그 후 선형 계층에서는 계산된 확률 값 중 최적의 값(최종 출력 단어)을 선택하기 위해 소프트맥스 활성화 함수를 사용합니다.

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(output_dim, self.embbed_dim) # 임베딩 계층 초기화
        self.gru = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers) # GRU 계층 초기화
        self.out = nn.Linear(self.hidden_dim, output_dim) # 선형 계층 초기화
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input = input.view(1, -1) # 입력을 (1, 배치 크기)로 변경
        embedded = F.relu(self.embedding(input))
        output, hidden = self.gru(embedded, hidden)
        prediction = self.softmax(self.out(output[0]))
        return prediction, hidden


* self.softmax = nn.LogSoftmax(dim=1)
    * 소프트맥스는 일정한 시퀀스의 숫자들을 0과 1 사이의 양의 수로 변환해서 클래스의 확률을 구할 때 사용합니다.
    * 로그 소프트맥스(LogSoftmax)는 소프트맥스와 로그 함수의 결합입니다.
    * 소프트맥스 활성화 함수에서 발생할 수 있는 `기울기 소멸 문제를 방지`하기 위해 만들어진 활성화 함수입니다.

앞에서 정의한 인코더와 디코더를 이용하여 seq2seq 모델을 정의합니다. 인코더와 디코더를 이용한 seq2seq 네트워크는 다음 그림과 같습니다.

![](../Static/575.jpg)


In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()

        self.encoder = encoder #인코더 초기화
        self.decoder = decoder # 디코더 초기화
        self.device = device

    def forward(self, input_lang, output_lang, teacher_forcing_ratio=0.5):
        input_length = input_lang.size(0) # 입력 문자 길이(문장의 단어 수)
        batch_size = output_lang.shape[1]
        target_length = output_lang.shape[0]
        vocab_size = self.decoder.output_dim
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device) # 예측된 출력을 저장하기 위한 변수 초기화

        for i in range(input_length):
            encoder_output, encoder_hidden = self.encoder(input_lang[i]) # 문장의 모든 단어를 인코딩
        decoder_hidden = encoder_hidden.to(device) # 인코더의 은닉층을 디코더의 은닉층으로 사용
        decoder_input = torch.tensor([SOS_token], device=device) # 첫 번째 예측 단어 앞에 토큰(sos) 추가

        for t in range(target_length): # 현재 단어에서 출력 단어를 예측
            decoder_output, decoder_hidden = self.decoder(
                decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            input = (output_lang[t] if teacher_force else topi) # teacher_force를 활성화 하면 목표 단어를 다음 입력으로 사용
            if (teacher_force == False and input.item() == EOS_token): # teacher_force를 활성화하지 않으면 자체 예측 값을 다음 입력으로 사용
                break
        return outputs


*  teacher_force = random.random() < teacher_forcing_ratio
    * 티처포스(teacher_force)는 seq2seq(인코더-디코더) 모델에서 많이 사용되는 기법입니다. 티처포스는 다음 그림과 같이 번역(예측)하려는 목표 단어(ground truth)를 디코더의 다음 입력으로 넣어 주는 기법입니다.

    ![](../Static/577.jpg)

티처포스를 사용하면 학습 초기에 `안정적인 훈련`이 가능하며, 기울기를 계산할 때 빠른 수렴이 가능한 장점이 있지만 `네트워크가 불안정해질 수 있는 단점`이 있습니다.

모델 훈련을 위한 함수를 정의합니다. 여기에서는 모델의 오차를 계산하는 부분만 정의합니다.

In [23]:
teacher_forcing_ratio = 0.5


def Model(model, input_tensor, target_tensor, model_optimizer,  criterion):
    model_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0
    output = model(input_tensor, target_tensor)
    num_iter = output.size(0)

    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot]) # 모델의 예측 결과와 정답(예상 결과)을 이용하여 오차를 계산
    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter
    return epoch_loss


In [24]:
def trainModel(model, input_lang, output_lang, pairs, num_iteration=20000):
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=0.01) # 옵티마이저로 SGD를 사용
    criterion = nn.NLLLoss()
    total_loss_iterations = 0

    training_pairs = [tensorsFromPair(
        input_lang, output_lang, random.choice(pairs)) for i in range(num_iteration)]

    for iter in range(1, num_iteration+1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = Model(model, input_tensor, target_tensor, optimizer, criterion)
        total_loss_iterations += loss

        if iter % 5000 == 0:
            average_loss = total_loss_iterations / 5000 # 5000번쨰마다 오차 값에 대해 출력
            total_loss_iterations = 0
            print('%d %.4f' % (iter, average_loss))

    torch.save(model.state_dict(), './mytraining.pt')
    return model


* criterion = nn.NLLLoss()
    * `NLLLoss` 역시 크로스엔트로피 손실 함수(CrossEntropyLoss)와 마찬가지로 `분류 문제`에 사용합니다. 이 둘간의 차이는 다음과 같습니다. 크로스엔트로피 손실 함수에는 `LogSoftmax + NLLLoss`가 포함되어 있습니다. 따라서 `크로스엔트로피 손실 함수를 사용할 경우에는 소프트맥스를 명시하지 않아도 되지만` NLLLoss를 사용할 때는 사용자가 소프트맥스를 사용할 것임을 명시해야 합닏. 이러한 이유로 모델 네트워크 부분에서도 소프트맥스 활성화 함수를 지정해 주었습니다.

이제 모델을 평가하기 위한 함수를 정의합니다.

In [25]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0]) # 입력 문자열을 텐서로 변환
        output_tensor = tensorFromSentence(output_lang, sentences[1]) # 출력 문자열을 텐서로 변환
        decoded_words = []
        output = model(input_tensor, output_tensor)

        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1) # 각 출력에서 가장 높은 값을 찾아 인덱스를 반환

            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>') # EOS 토큰을 만나면 평가를 멈춤
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()]) # 예측 결과를 출력 문자열에 추가

    return decoded_words


def evaluateRandomly(model, input_lang, output_lang, pairs, n=10): # 훈련 데이터셋으로부터 임의의 문장을 가져와서 모델 평가
    for i in range(n):
        pair = random.choice(pairs) # 임의의 문장을 가져옴
        print('input {}'.format(pair[0]))
        print('output {}'.format(pair[1]))
        output_words = evaluate(model, input_lang, output_lang, pair) # 모델 평가 결과는 output_words에 저장
        output_sentence = ' '.join(output_words)
        print('predicted {}'.format(output_sentence))


In [26]:
lang1 = 'eng' # 입력으로 사용할 영어
lang2 = 'fra' # 출력으로 사용할 프랑스어
input_lang, output_lang, pairs = process_data(lang1, lang2)

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

input_size = input_lang.n_words
output_size = output_lang.n_words
print('Input : {} Output : {}'.format(input_size, output_size)) # 입력과 출력에 대한 단어 수 출력

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 5001

encoder = Encoder(input_size, hidden_size, embed_size, num_layers) # 인코더에 훈련 데이터셋을 입력하고 모든 출력과 은닉 상태를 저장
# 디코더의 첫 번째 입력으로 <SOS> 토큰이 제공되고, 인코더의 마지막 은닉 상태가 디코더의 첫 번째 은닉 상태로 제공됩니다.
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)
model = Seq2Seq(encoder, decoder, device).to(device)  # 인코더-디코더 모델(seq2seq)의 객체 생성

print(encoder)
print(decoder)

model = trainModel(model, input_lang, output_lang, pairs, num_iteration) # 모델 학습

C:\Users\aqs45\AppData\Local\Temp\ipykernel_5308\3068071628.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  sentence = sentence.str.replace('[^A-Za-z\s]+', ' ')


random sentence ['i figured that tom would never find it ', 'j ai suppos  que tom n allait jamais le trouver ']
Input : 14869 Output : 21050
Encoder(
  (embedding): Embedding(14869, 256)
  (gru): GRU(256, 512)
)
Decoder(
  (embedding): Embedding(21050, 256)
  (gru): GRU(256, 512)
  (out): Linear(in_features=512, out_features=21050, bias=True)
  (softmax): LogSoftmax(dim=1)
)
5000 4.9130


seq2seq는 `인코더와 디코더 네트워크`를 사용합니다. 하지만 일반적인 seq2seq 모델은 입력 문장이 긴 시퀀스일 경우 정확한 처리가 어렵습니다. 즉, 다음과 같은 이유로 어텐션 매커니즘(attention mechanism)이 등장했습니다.

* 하나의 고정된 크기의 벡터에 모든 정보를 담다 보니 정보의 손실 발생
* RNN에서 발생할 수 있는 기울기 소멸(vanishing gradient) 문제 발생

어텐션 메커니즘은 그림의 (b)와 같이 디코딩 프로세스 동안 입력 시퀀스의 모든 숨겨진 상태를 유지하고 활용하기 때문에 정보의 손실과 기울기 소멸 문제가 발생하지 않습니다. 즉 (b)와 같이 컨텍스트 벡터는 인코더의 전체 은닉 상태들과 디코더의 이전 은닉상태를 바탕으로 만들어집니다.

![](../Static/582.jpg)

정리하면 어텐션 메커니즘이 쓰이지 않은 기존 모델과 다른 점은 디코더에서 컨텍스트 벡터가 모두 같은 것을 쓰거나 단순히 전파되는 것이 아니라 특정 시점(time step)마다 다른 컨텍스트 벡터를 사용한다는 것입니다.

다음 그림은 코드에서 사용되는 모델의 네트워크를 도식화한 것입니다. 모델과 비교를 위해 네트워크의 영문명을 그대로 사용했습니다.


![](../Static/583.jpg)



In [27]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_langth = MAX_LENGTH

        self.embedding = nn.Embedding(self.output_size, self.hidden_size) # 임베딩 계층 초기화
        self.attn = nn.Linear(self.hidden_size * 2, self.max_langth)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).viwe(1,1,-1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

* self.attn = nn.Linear(self.hidden_size * 2, self.max_langth)
    * 어텐션은 입력을 디코더로 변환합니다. 즉, 어텐션은 입력 시퀀스와 길이가 같은 인코딩된 시퀀스로 변환하는 역할을 합니다. 따라서 'self.max_length'는 모든 입력 시퀀스의 최대 길이여야 합니다.
    * `torch.bmm` 함수는 배치 행렬 곱(Batch Matrix Multiplication, BMM)을 수행하는 함수로, 다음 그림과 같이 두 개 이상의 차원을 지닌 텐서가 주어졌을때 뒤의 두 개의 차원에 대해 행렬 곱을 수행하는 함수입니다. 예를 들어 (BN, A, B)와 (BN, B, C)라는 행렬이 두 개 주어졌을 때 각 행렬에서 뒤 두개의 차원((A,B),(B,C))에 대해 행렬 곱을 수행하면 (BN,A,C)라는 결과를 얻을 수 있습니다.
    * ![](../Static/584.jpg)
    * 따라서 `torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))`은 가중치와 인코더의 출력 벡터를 곱하겠다는 의미이며, 그 결과(attn_applied)는 입력 시퀀스의 특정 부분에 관한 정보를 포함하고 있기 때문에 디코더가 적절한 출력 단어를 선택하도록 도와줍니다.

인코더와 어텐션 디코더를 이용하여 모델을 학습시키기 위한 함수를 정의합니다.

In [28]:
import time

lang1 = 'eng' # 입력으로 사용할 영어
lang2 = 'fra' # 출력으로 사용할 프랑스어
input_lang, output_lang, pairs = process_data(lang1, lang2)

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=1e-2):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_paris = [tensorsFromPair(input_lang, output_lang, random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters+1):
        training_paris = training_paris[iter-1]
        input_tensor = training_paris[0]
        target_tensor = training_paris[1]
        loss = Model(model, input_tensor, target_tensor, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

    if iter % 5000 == 0:
        print_loss_avg = print_loss_total / 5000
        print_loss_total = 0
        print('%d, %.4f' % (iter, print_loss_avg))
    

C:\Users\aqs45\AppData\Local\Temp\ipykernel_5308\3068071628.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  sentence = sentence.str.replace('[^A-Za-z\s]+', ' ')
